# Data Science Project

This notebook contains a template for a data science project with common sections and workflows.

## Project Overview
- **Objective**: Define your project goals here
- **Dataset**: Describe your dataset
- **Methodology**: Outline your approach

## 1. Import Libraries

Import all necessary libraries for data manipulation, visualization, and modeling.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Settings
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Random seed for reproducibility
np.random.seed(42)

## 2. Data Loading

Load your dataset and perform initial inspection.

In [ ]:
# Load data
# df = pd.read_csv('your_data.csv')
# For demonstration, let's create a sample dataset
df = pd.DataFrame({
    'feature_1': np.random.randn(100),
    'feature_2': np.random.randn(100),
    'feature_3': np.random.randint(0, 10, 100),
    'target': np.random.randint(0, 2, 100)
})

print(f"Dataset shape: {df.shape}")
df.head()

## 3. Exploratory Data Analysis (EDA)

Understand the structure, distributions, and relationships in your data.

In [ ]:
# Basic information
print("Dataset Info:")
df.info()

print("\nBasic Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

if missing_values.sum() == 0:
    print("No missing values found.")

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

### 3.1 Data Visualization

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions', fontsize=16)

for idx, col in enumerate(df.columns[:4]):
    row = idx // 2
    col_idx = idx % 2
    axes[row, col_idx].hist(df[col], bins=30, edgecolor='black')
    axes[row, col_idx].set_title(f'Distribution of {col}')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Pairplot for feature relationships
sns.pairplot(df, hue='target', diag_kind='kde', height=2.5)
plt.suptitle('Pairplot of Features', y=1.02, fontsize=14)
plt.show()

## 4. Data Preprocessing

Prepare the data for modeling by handling missing values, encoding, and scaling.

In [ ]:
# Handle missing values (if any)
# df.fillna(df.mean(), inplace=True)
# or
# df.dropna(inplace=True)

print("Data preprocessing completed.")
print(f"Final dataset shape: {df.shape}")

In [ ]:
# Encode categorical variables (if needed)
# df_encoded = pd.get_dummies(df, columns=['categorical_column'], drop_first=True)

# Feature engineering
# Create new features based on domain knowledge
# df['new_feature'] = df['feature_1'] * df['feature_2']

## 5. Feature Selection & Split

Separate features and target, then split into training and testing sets.

In [ ]:
# Separate features and target
X = df.drop('target', axis=1)
y = df['target']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed.")

## 6. Model Development

Build and train machine learning models.

In [ ]:
# Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

In [ ]:
# Train and evaluate models
results = {}

for name, model in models.items():
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"\n{name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

## 7. Model Evaluation

Evaluate and compare model performance.

In [ ]:
# Compare model accuracies
plt.figure(figsize=(10, 6))
models_list = list(results.keys())
accuracies = list(results.values())

plt.bar(models_list, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.xlabel('Models', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Model Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, 1])

# Add value labels on bars
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nBest Model: {max(results, key=results.get)} with accuracy: {max(results.values()):.4f}")

In [ ]:
# Confusion matrix for the best model
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
y_pred_best = best_model.predict(X_test_scaled)

cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True, 
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.show()

## 8. Feature Importance

Analyze which features are most important for the model.

In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'], feature_importance['importance'])
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title('Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nFeature Importance:")
    print(feature_importance)
else:
    print(f"{best_model_name} does not support feature importance.")

## 9. Model Tuning (Optional)

Fine-tune hyperparameters using Grid Search or Random Search.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Example: Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Uncomment to run grid search (can be time-consuming)
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_train_scaled, y_train)

# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

print("Hyperparameter tuning section (uncomment to run).")

## 10. Conclusions

Summarize your findings and next steps.

### Key Findings:
- List your main discoveries from the analysis
- Highlight model performance metrics
- Discuss feature importance insights

### Recommendations:
- Provide actionable recommendations based on your analysis
- Suggest areas for further investigation

### Next Steps:
- Deploy the best model
- Collect more data if needed
- Try advanced techniques (ensemble methods, deep learning, etc.)
- Monitor model performance in production

## 11. Save Model (Optional)

Save the trained model for future use.

In [ ]:
import pickle

# Save the best model
# with open('best_model.pkl', 'wb') as f:
#     pickle.dump(best_model, f)

# Save the scaler
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

print("Model saving section (uncomment to save).")